In [10]:
import spotipy
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [11]:
load_dotenv()

True

In [12]:
scope = ['ugc-image-upload'
        ,'user-read-recently-played'
        ,'user-top-read'
        ,'user-read-playback-position'
        ,'user-read-playback-state'
        ,'user-modify-playback-state'
        ,'user-read-currently-playing'
        ,'app-remote-control'
        ,'streaming'
        ,'playlist-modify-public'
        ,'playlist-modify-private'
        ,'playlist-read-private'
        ,'playlist-read-collaborative'
        ,'user-follow-modify'
        ,'user-follow-read'
        ,'user-library-modify'
        ,'user-library-read'
        ,'user-read-email'
        ,'user-read-private'
        ]


auth_manager = spotipy.oauth2.SpotifyOAuth(scope=scope,
                                            show_dialog=True)

# auth_manager.get_access_token(request.args.get("code"))
auth_url = auth_manager.get_authorize_url()
spotify = spotipy.Spotify(auth_manager=auth_manager)


In [13]:
artists_items = spotify.current_user_top_artists(limit=100, time_range='long_term')['items']

In [116]:
temp = pd.DataFrame.from_dict(artists_items)

In [21]:
def clean_top_artists(items_df):
    '''
        
    '''

    items_df['external_urls'] = temp['external_urls'].map(lambda x: x['spotify'])
    items_df['images'] = temp['images'].map(lambda x: x[0]['url'])
    items_df['followers'] = temp['followers'].map(lambda x: x['total'])

    return items_df

In [87]:
def clean_top_tracks(tracks_df):
    '''
        
    '''
    tracks_df['album_name'] = temp['album'].map(lambda x:x['name'])
    tracks_df['album_art'] = temp['album'].map(lambda x:x['images'][0]['url'])
    tracks_df['album_href'] = temp['album'].map(lambda x:x['href'])
    tracks_df['album_release_date'] = temp['album'].map(lambda x:x['release_date'])

    tracks_df['artist_names'] = temp['artists'].map(lambda x: ', '.join([a['name'] for a in x]))

    drop_cols = ['album', 'artists', 'available_markets', 'disc_number', 'external_ids', 'is_local', 'external_urls',\
                 'track_number', 'duration_ms', 'episode', 'track', 'uri',\
                     'preview_url', 'type',	'album_name', 'album_href','album_release_date' ]
    tracks_df = tracks_df.drop(drop_cols, axis=1)

    return tracks_df

In [73]:
def clean_playlist(user_playlists, playlist):
    '''

    '''

    filter_playlist = [i for i in spotify.current_user_playlists()['items'] if i['name']==playlist_name][0]
    filter_id = filter_playlist['id']
    playlist_filter_id = spotify.playlist(filter_id)
    playlist_tracks = playlist_filter_id['tracks']
    list_tracks = [playlist_tracks['items'][i]['track'] for i in range(filter_playlist['tracks']['total'])]
    temp = pd.DataFrame.from_dict(list_tracks)
    temp_df = clean_top_tracks(temp)

    return temp_df

In [72]:
def audio_playlist_features(playlist_df):
    '''

    '''
    df = pd.DataFrame.from_dict([spotify.audio_features(playlist_df['id'][i])[0] for i in range(len(playlist_df))])
    drop_cols = ['analysis_url', 'duration_ms', 'time_signature', 'uri', 'track_href', 'type']
    audio_df = df.drop(drop_cols, axis=1)

    return audio_df

In [192]:
term = 'short'

In [193]:
track_items =spotify.current_user_top_tracks(limit=100, time_range=f'{term}_term')['items']

In [194]:
temp = pd.DataFrame.from_dict(track_items)

In [195]:
temp_df = clean_top_tracks(temp)

In [196]:
len(temp_df)

50

In [197]:
temp_df.to_csv(f'top_tracks_{term}.csv')

In [3]:
playlists= ['229Jams',
 'Lorem',
 'RapCaviar',
 'Discover Weekly',
 'Hindi Songs - Mega Hit Mix - Bollywood songs - All time top 1000 songs',
 'Coffee Table Jazz',
 'Body Radio',
 'Classical Upbeat Piano',
 'Best of the Decade For You',
 'Your Top Songs 2019',
 'Hot Rhythmic',
 'lofi hip hop music - beats to relax/study to',
 'Late Night Jazz',
 'Liked from Radio',
 'Kimi No Nawa (Your Name) OST | RADWIMPS 💫 ',
 'Moood🤙',
 'All Out 10s',
 'Hamilton!']

In [9]:
playlists_kv = [dict([('label', k), ('value', k)]) for k in playlists]
playlists_kv

[{'label': '229Jams', 'value': '229Jams'},
 {'label': 'Lorem', 'value': 'Lorem'},
 {'label': 'RapCaviar', 'value': 'RapCaviar'},
 {'label': 'Discover Weekly', 'value': 'Discover Weekly'},
 {'label': 'Hindi Songs - Mega Hit Mix - Bollywood songs - All time top 1000 songs',
  'value': 'Hindi Songs - Mega Hit Mix - Bollywood songs - All time top 1000 songs'},
 {'label': 'Coffee Table Jazz', 'value': 'Coffee Table Jazz'},
 {'label': 'Body Radio', 'value': 'Body Radio'},
 {'label': 'Classical Upbeat Piano', 'value': 'Classical Upbeat Piano'},
 {'label': 'Best of the Decade For You',
  'value': 'Best of the Decade For You'},
 {'label': 'Your Top Songs 2019', 'value': 'Your Top Songs 2019'},
 {'label': 'Hot Rhythmic', 'value': 'Hot Rhythmic'},
 {'label': 'lofi hip hop music - beats to relax/study to',
  'value': 'lofi hip hop music - beats to relax/study to'},
 {'label': 'Late Night Jazz', 'value': 'Late Night Jazz'},
 {'label': 'Liked from Radio', 'value': 'Liked from Radio'},
 {'label': 'Ki

In [149]:
user_playlists = [i['name'] for i in spotify.current_user_playlists()['items']]
len(user_playlists)

18

In [30]:
playlist_name = 'Kimi No Nawa (Your Name) OST | RADWIMPS 💫 '

In [14]:
# playlist_df.to_csv('playlist.csv')

In [15]:
# spotify.current_user_recently_played?

In [16]:
# spotify.current_user_recently_played()['items']

In [387]:
# def get_features(sp, track_id):
#     '''
#     Returns a dictionary of features features that is used in KNN model
    
#     artist: Artist for which features are needed - str
#     sp: Spotify Token - spotipy.client.Spotify
#     features: List of features to return - List
#     '''    
#     d = dict()
#     audio_features = sp.audio_features(track_id)[0]
#     for feats in features:
#         if feats not in d.keys():
#             if feats in audio_features:
#                 d[feats] = audio_features[feats]
#     return d

In [396]:
playlist_features = audio_playlist_features(playlist_df)

In [397]:
# playlist_features.to_csv('playlist_features.csv')

In [33]:
# spotify.current_user_saved_tracks(20)['items'][:2]

In [35]:
user_playlists = [i['name'] for i in spotify.current_user_playlists()['items']]
user_playlists

['229Jams',
 'Lorem',
 'RapCaviar',
 'Discover Weekly',
 'Hindi Songs - Mega Hit Mix - Bollywood songs - All time top 1000 songs',
 'Coffee Table Jazz',
 'Body Radio',
 'Classical Upbeat Piano',
 'Best of the Decade For You',
 'Your Top Songs 2019',
 'Hot Rhythmic',
 'lofi hip hop music - beats to relax/study to',
 'Late Night Jazz',
 'Liked from Radio',
 'Kimi No Nawa (Your Name) OST | RADWIMPS 💫 ',
 'Moood🤙',
 'All Out 10s',
 'Hamilton!']

In [108]:
final_df = pd.DataFrame()
for playlist_name in user_playlists:
    
    filter_playlist = [i for i in spotify.current_user_playlists()['items'] if i['name']==playlist_name][0]
    if filter_playlist['tracks']['total'] <5:
        continue
    print(playlist_name, filter_playlist['tracks']['total'])
    filter_id = filter_playlist['id']
    playlist_filter_id = spotify.playlist(filter_id)
    playlist_tracks = playlist_filter_id['tracks']
    list_tracks = [playlist_tracks['items'][i]['track'] for i in range(min(10, filter_playlist['tracks']['total']))]
    temp = pd.DataFrame.from_dict(list_tracks)
    # audio_playlist_features(clean_top_tracks(temp))
    c_tracks = clean_top_tracks(temp)
    audio_features = audio_playlist_features(c_tracks)
    merged_inner = pd.merge(left=c_tracks, right=audio_features, left_on='id', right_on='id')
    merged_inner['playlist_name'] = playlist_name
    # Stack the DataFrames on top of each other
    final_df = pd.concat([final_df, merged_inner], axis=0)

Lorem 178
RapCaviar 50
Discover Weekly 30
Hindi Songs - Mega Hit Mix - Bollywood songs - All time top 1000 songs 813
Coffee Table Jazz 149
Body Radio 50
Classical Upbeat Piano 12
Best of the Decade For You 100
Your Top Songs 2019 100
Hot Rhythmic 83
lofi hip hop music - beats to relax/study to 300
Late Night Jazz 100
Liked from Radio 5
Kimi No Nawa (Your Name) OST | RADWIMPS 💫  41
Moood🤙 21
All Out 10s 150
Hamilton! 36


In [ ]:
def get_tsne_csv(spotify, min_songs_per_playlist=5, k=10):
    '''
    '''
    user_playlists = [i['name'] for i in spotify.current_user_playlists()['items']]
    final_df = pd.DataFrame()
    for playlist_name in user_playlists:
        
        filter_playlist = [i for i in spotify.current_user_playlists()['items'] if i['name']==playlist_name][0]
        if filter_playlist['tracks']['total'] <min_songs_per_playlist:
            continue
        # print(playlist_name, filter_playlist['tracks']['total'])
        filter_id = filter_playlist['id']
        playlist_filter_id = spotify.playlist(filter_id)
        playlist_tracks = playlist_filter_id['tracks']
        list_tracks = [playlist_tracks['items'][i]['track'] for i in range(min(10, filter_playlist['tracks']['total']))]
        temp = pd.DataFrame.from_dict(list_tracks)

        c_tracks = clean_top_tracks(temp)
        audio_features = audio_playlist_features(c_tracks)
        merged_inner = pd.merge(left=c_tracks, right=audio_features, left_on='id', right_on='id')
        merged_inner['playlist_name'] = playlist_name

        # Stack the DataFrames on top of each other
        final_df = pd.concat([final_df, merged_inner], axis=0)

    # K Means - Predicted Genre feature 
    X = copy.deepcopy(final_df)
    song_features = pd.DataFrame()
    scaler = MinMaxScaler() # normalizer instance
    for col in X.columns: 
        if col not in ['artists','predicted_genres', 'href', 'id', 'name', 'playlist_name',  'album_art', 'artist_names',]:
            scaler.fit(X[[col]])
            song_features[col] = scaler.transform(X[col].values.reshape(-1,1)).ravel() 

    km = KMeans(n_clusters=k)
    predicted_genres = km.fit_predict(song_features)
    X['predicted_genres'] = predicted_genres

    return X

In [109]:
len(final_df)

165

In [110]:
final_df.head()

,explicit,href,id,name,popularity,album_art,artist_names,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,playlist_name
0,True,https://api.spotify.com/v1/tracks/1QL7nSDZCwZM...,1QL7nSDZCwZMnbisV4KOXt,t r a n s p a r e n t s o u l feat. Travis Barker,75,https://i.scdn.co/image/ab67616d0000b273e76af6...,"WILLOW, Travis Barker",0.390,0.830,11,-5.201,0,0.0563,0.00210,0.000000,0.156,0.191,90.021,Lorem
1,False,https://api.spotify.com/v1/tracks/26tMEDbutzry...,26tMEDbutzryy8RyHX8HX4,From These Heights,48,https://i.scdn.co/image/ab67616d0000b273336b84...,Jelani Aryeh,0.758,0.903,4,-3.909,1,0.0314,0.00949,0.000007,0.628,0.749,125.002,Lorem
2,False,https://api.spotify.com/v1/tracks/6xpDh0dXrkVp...,6xpDh0dXrkVp0Po1qrHUd8,Adam's Song,70,https://i.scdn.co/image/ab67616d0000b2736da502...,blink-182,0.520,0.775,0,-5.803,1,0.0348,0.03400,0.000000,0.364,0.382,136.028,Lorem
3,True,https://api.spotify.com/v1/tracks/0h7Vz5MQyJjO...,0h7Vz5MQyJjO4ZnMW7HTjV,Off the Rails,45,https://i.scdn.co/image/ab67616d0000b273cc66c7...,Wallice,0.476,0.798,1,-4.208,1,0.0394,0.05810,0.001800,0.276,0.738,168.179,Lorem
4,False,https://api.spotify.com/v1/tracks/3f9Xt5Bcu1Qo...,3f9Xt5Bcu1QooplxlTPEtb,Ur Gonna Wish U Believed Me,51,https://i.scdn.co/image/ab67616d0000b273caed83...,Cavetown,0.568,0.453,4,-7.453,1,0.0258,0.62600,0.022400,0.135,0.224,134.996,Lorem


In [118]:
# from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans 
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import joblib
import plotly.graph_objects as go
import copy
import matplotlib.pyplot as plt

%matplotlib inline


In [135]:
# Preprocessing data
X = copy.deepcopy(final_df)
song_features = pd.DataFrame()
scaler = MinMaxScaler() # normalizer instance

In [136]:
final_df.columns

Index(['explicit', 'href', 'id', 'name', 'popularity', 'album_art',
       'artist_names', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'playlist_name'],
      dtype='object')

In [137]:
for col in X.columns: 
    if col not in ['artists','predicted_genres', 'href', 'id', 'name', 'playlist_name',  'album_art', 'artist_names',]:
        scaler.fit(X[[col]])
        song_features[col] = scaler.transform(X[col].values.reshape(-1,1)).ravel() 

In [148]:
# # KMeans instance
# km = KMeans()
# k_rng = range(1,100)  # k value
# sse = [] # sse value for each k
# for i in k_rng:
#     km = KMeans(n_clusters = i)
#     km.fit(song_features.sample(100))
#     sse.append(km.inertia_) # calculating sse
# plt.plot(k_rng,sse)
# plt.xlabel('K value')
# plt.ylabel('SSE Error')
# plt.title('Best K value')
# plt.show()

In [139]:
# Train for k = 10
np.random.seed(3)
k = 10
km = KMeans(n_clusters=k)
predicted_genres = km.fit_predict(song_features)

In [140]:
song_features['predicted_genres'] = predicted_genres

In [144]:
X.shape, song_features.shape

((165, 19), (165, 14))

In [146]:
X['predicted_genres'] = predicted_genres

In [147]:
X

,explicit,href,id,name,popularity,album_art,artist_names,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,playlist_name,predicted_genres
0,True,https://api.spotify.com/v1/tracks/1QL7nSDZCwZM...,1QL7nSDZCwZMnbisV4KOXt,t r a n s p a r e n t s o u l feat. Travis Barker,75,https://i.scdn.co/image/ab67616d0000b273e76af6...,"WILLOW, Travis Barker",0.390,0.830,11,-5.201,0,0.0563,0.00210,0.000000,0.1560,0.191,90.021,Lorem,3
1,False,https://api.spotify.com/v1/tracks/26tMEDbutzry...,26tMEDbutzryy8RyHX8HX4,From These Heights,48,https://i.scdn.co/image/ab67616d0000b273336b84...,Jelani Aryeh,0.758,0.903,4,-3.909,1,0.0314,0.00949,0.000007,0.6280,0.749,125.002,Lorem,7
2,False,https://api.spotify.com/v1/tracks/6xpDh0dXrkVp...,6xpDh0dXrkVp0Po1qrHUd8,Adam's Song,70,https://i.scdn.co/image/ab67616d0000b2736da502...,blink-182,0.520,0.775,0,-5.803,1,0.0348,0.03400,0.000000,0.3640,0.382,136.028,Lorem,7
3,True,https://api.spotify.com/v1/tracks/0h7Vz5MQyJjO...,0h7Vz5MQyJjO4ZnMW7HTjV,Off the Rails,45,https://i.scdn.co/image/ab67616d0000b273cc66c7...,Wallice,0.476,0.798,1,-4.208,1,0.0394,0.05810,0.001800,0.2760,0.738,168.179,Lorem,0
4,False,https://api.spotify.com/v1/tracks/3f9Xt5Bcu1Qo...,3f9Xt5Bcu1QooplxlTPEtb,Ur Gonna Wish U Believed Me,51,https://i.scdn.co/image/ab67616d0000b273caed83...,Cavetown,0.568,0.453,4,-7.453,1,0.0258,0.62600,0.022400,0.1350,0.224,134.996,Lorem,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,True,https://api.spotify.com/v1/tracks/0LpHC9mhPAQC...,0LpHC9mhPAQC98IjXZIrif,The Election of 1800,62,https://i.scdn.co/image/ab67616d0000b273d72fb5...,"Daveed Diggs, Okieriete Onaodowan, Leslie Odom...",0.731,0.420,10,-10.199,1,0.4620,0.31900,0.000000,0.3060,0.645,67.426,Hamilton!,5
6,True,https://api.spotify.com/v1/tracks/0clwOjS9p20c...,0clwOjS9p20cEIQkuB7bEP,An Open Letter (feat. Shockwave) - Interlude,44,https://i.scdn.co/image/ab67616d0000b27382288d...,"Watsky, Shockwave",0.728,0.875,11,-7.828,0,0.8520,0.47800,0.000000,0.6310,0.591,75.279,Hamilton!,3
7,False,https://api.spotify.com/v1/tracks/6SHI6STEW51c...,6SHI6STEW51cQkAXBRpLNj,Your Obedient Servant,62,https://i.scdn.co/image/ab67616d0000b273d72fb5...,"Leslie Odom Jr., Lin-Manuel Miranda, Original ...",0.736,0.370,5,-11.072,1,0.5020,0.29300,0.000000,0.0420,0.619,165.908,Hamilton!,1
8,True,https://api.spotify.com/v1/tracks/1CzeuSrm71wH...,1CzeuSrm71wHP9qsjg7p3F,The Story of Tonight - Reprise,66,https://i.scdn.co/image/ab67616d0000b273d72fb5...,"Anthony Ramos, Okieriete Onaodowan, Daveed Dig...",0.724,0.305,5,-12.763,0,0.2570,0.68000,0.000000,0.1280,0.594,81.874,Hamilton!,3
